In [1]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import re as re
import os
import copy
import statsmodels.formula.api as sm
import pymc3 as pm
import matplotlib.pyplot as plt
from tqdm import tqdm

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\Yurii\Anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
drc = pd.read_excel('Data\\south_of_sahara\\DRC_Conflict_27June2012.xlsx', header = 0, index_col = 1)
uganda = pd.read_excel('Data\\south_of_sahara\\Uganda_Conflict_27June2012.xlsx', header = 0, index_col = 1)
ethiopia = pd.read_excel('Data\\south_of_sahara\\Ethiopia_Conflict_27June2012.xlsx', header = 0, index_col = 1)
sudan = pd.read_excel('Data\\south_of_sahara\\Sudan_Conflict_27June2012.xlsx', header = 0, index_col = 1)
burundi = pd.read_excel('Data\\south_of_sahara\\Burundi_Conflict_20June2012.xlsx', header = 0, index_col = 1)

ethiopia.drop(ethiopia[ethiopia['precision'] == '.'].index, inplace = True)
burundi.drop(burundi[burundi['precision'] == '        .'].index, inplace = True)

all_provinces = gpd.read_file('Data\\Shapefiles\\ADM1\\compiled.shp')

drc = drc[drc['precision'] <=6]
uganda = uganda[uganda['precision'] <=6]
ethiopia = ethiopia[ethiopia['precision'] <=6]
sudan = sudan[sudan['precision'] <=6]
burundi = burundi[burundi['precision'] <=6]

worker_deaths = pd.read_csv('Data\\security_incidents2018-08-02.csv', encoding = "ISO-8859-1")
conflict = pd.read_csv('Data\\conflict_data\\gedevents-2018-07-13.csv', header = 0, index_col = 0)

geometry = [Point(xy) for xy in zip(conflict.longitude, conflict.latitude)]
gconflict = gpd.GeoDataFrame(conflict, crs = {'init': 'epsg:4326'}, geometry = geometry)

geometry = [Point(xy) for xy in zip(worker_deaths.Longitude, worker_deaths.Latitude)]
g_w_d = gpd.GeoDataFrame(worker_deaths, crs = {'init': 'epsg:4326'}, geometry = geometry)

gconflict = gpd.sjoin(gconflict, all_provinces, how="inner")
g_w_d = gpd.sjoin(g_w_d, all_provinces, how="inner")

In [3]:
def recode_commitments(df):
    """
        Commitments are coded for for all locations within a project. 
        To arrive at an average per location divide the value over numbloc
    """
    df['usdcr'] = df['usdcr'] / df['numbloc']
    df['usdco'] = df['usdco'] / df['numbloc']
    return df

def geocode(df):
    geometry = [Point(xy) for xy in zip(df.long, df.lat)]
    gdf = gpd.GeoDataFrame(df, crs = {'init': 'epsg:4326'}, geometry = geometry)
    gaid = gpd.sjoin(gdf, all_provinces, how="inner")
    return gaid

In [4]:
countries = [drc, uganda, ethiopia, sudan, burundi]
geoaid = [geocode(country) for country in countries]
geoaid = list(map(recode_commitments, geoaid))

geoaid_df = pd.DataFrame()
for country in geoaid:
    geoaid_df = geoaid_df.append(country)

In [5]:
geoaid_df.loc[:, 'dname'] = geoaid_df['dname'].apply(lambda x: x.replace(" ", ""))
geoaid_df.loc[geoaid_df['dname'] == 'BELGIU', 'dname'] = 'BELGIUM'
geoaid_df.loc[geoaid_df['dname'] == 'FINLAN', 'dname'] = 'FINLAND'
geoaid_df.loc[geoaid_df['dname'] == 'IRELAN', 'dname'] = 'IRELAND'
geoaid_df.loc[geoaid_df['dname'] == 'PORTUG', 'dname'] = 'PORTUGAL'
geoaid_df.loc[geoaid_df['dname'] == 'GERMAN', 'dname'] = 'GERMANY'
geoaid_df.loc[geoaid_df['dname'] == 'AUSTRI', 'dname'] = 'AUSTRIA'
geoaid_df.loc[geoaid_df['dname'] == 'DENMAR', 'dname'] = 'DENMARK'
geoaid_df.loc[geoaid_df['dname'] == 'SWITZE', 'dname'] = 'SWITZERLAND'
geoaid_df.loc[geoaid_df['dname'] == 'LUXEMB', 'dname'] = 'LUXEMBOURG'
geoaid_df.loc[geoaid_df['dname'] == 'AUSTRA', 'dname'] = 'AUSTRALIA'
geoaid_df.loc[geoaid_df['dname'] == 'NETHER', 'dname'] = 'NETHERLANDS'

In [6]:
geoaid_df['crspcode'] =  geoaid_df['crspcode'].astype(str)
geoaid_df = geoaid_df.dropna(subset = ['usdco', 'crspcode'])
geoaid_df = geoaid_df[~geoaid_df.index.duplicated(keep='first')]

geoaid_df['fungible'] = 0
geoaid_df['nonfungible'] = 0

In [7]:
for i in geoaid_df.index:
    try:
        if geoaid_df.loc[i, 'crspcode'][:3] in ['311', '110', '230', '510']:
            geoaid_df.at[i, 'fungible'] = geoaid_df.loc[i, 'usdco']
        else:
            geoaid_df.at[i, 'nonfungible'] = geoaid_df.loc[i, 'usdco']
    except:
        print(i)

In [8]:
# Func calculates the coefficient of unalikeability (as defined by Kader 2007) of every sublcass of first_group variable by topic_name
# E.g. unalikeability of project location for each aid donor -> calc_unalikeability(gaid, 'donors', 'ad_sector_names')

def calc_unalikeability(data, first_group, topic_name):
    
    # prepare data: group by first_group and topic_name, and divide by the size of the respective group
    # thus we obtain the share that each topic_name has in the respective first_group
    
    d = (data.groupby([first_group, topic_name]).size() / data.groupby([first_group]).size())
    
    # here we get the keys for the first level grouping. So, the unique values of first group column
    keys = []
    for i in d.index:
        keys.append(i[0])
    keys = set(keys)
    
    # here we calculate the actual coefficient
    # for every value of first_group we calculate its coefficient:
    # coefficient is defined as 1 - SUM_i(p_i^2), where p_i is the share of the ith subgroup in the total group. 
    
    coefs = {}
    for key in keys:
        s = 0
        for subgroup in d[key]:
            s += subgroup ** 2
        coef = 1 - s
        coefs[key] = coef
    return pd.Series(coefs)


# calculates the variablity of topic_name (e.g. total commitments of money) for every member of first_group
def calc_var(data, first_group, topic_name):
    d = data.groupby(first_group)
    
    def var(d):
        d = d[topic_name]
        d = (d - min(d)) / (max(d) - min(d))
        
        if np.isnan(np.var(d)):
            return 0
        
        return np.var(d)
    
    return d.apply(var)

In [9]:
adaptability_by_focus = calc_unalikeability(geoaid_df, 'dname', 'crspcode')
adaptability_by_location =calc_unalikeability(geoaid_df, 'dname', 'NAME_1')
adaptability_by_start_year = calc_unalikeability(geoaid_df, 'dname', 'year')
adaptability_by_commitment = calc_var(geoaid_df, 'dname', 'usdco')
composite_adaptability = adaptability_by_commitment + adaptability_by_start_year + adaptability_by_location + adaptability_by_focus
median = composite_adaptability.median()
high_adaptability = composite_adaptability[composite_adaptability >= median]
low_adaptability = composite_adaptability[composite_adaptability < median]

In [10]:
# Compute the year, ADM1 pairs of events when aid workers were killed. 
# Then, we can assume that these are unreceptive year, region pairs.
# However, we can also divide this into 2 subgroups: more than average (or median) casualties vs less than average (or median) casualties
worker_deaths_grouped = g_w_d.groupby(['Year', 'NAME_1'])['Total affected'].sum()
worker_deaths_grouped.to_csv('measures_indices\\year_location_pairs.csv')

# The idea is that low_receptivity is depcited by a lot of worker deaths
# high receptivity is depicted by a no worker deaths


In [11]:
# custom function to check whether a tuple is included in an array of tuples (for some reason, basic function did not work)
def check_in (tupl, array):
    for element in array:
        if (tupl[0] == element[0]) & (tupl[1] == element[1]):
            return True

# select those row numbers (indices) of gaid dataframe that represent projects that started in the same year/adm1 combination
# as highly receptive ones
low_receptive_indices = []

low_receptive = pd.DataFrame()
highly_receptive = pd.DataFrame()


for index, row in tqdm(geoaid_df.iterrows()):
    tupl = (int(row['year']), row['NAME_1'])
    if check_in(tupl, worker_deaths_grouped.index.values):
        low_receptive_indices.append(index)
        low_receptive = low_receptive.append(row)
    else:
        highly_receptive = highly_receptive.append(row)

10799it [02:17, 78.42it/s]


In [12]:
# High receptivity + high adaptability indices
high_receptivity_high_adaptability = highly_receptive[highly_receptive['dname'].isin(high_adaptability.index)]
high_receptivity_low_adaptability = highly_receptive[highly_receptive['dname'].isin(low_adaptability.index)]

low_receptivity_high_adaptability = low_receptive[low_receptive['dname'].isin(high_adaptability.index)]
low_receptivity_low_adaptability = low_receptive[low_receptive['dname'].isin(low_adaptability.index)]

In [13]:
def add_lag(df, column_name, new_name):
    output = pd.DataFrame()
    df[new_name] = df.groupby(level = 0)[column_name].shift(1)
    by_country = df.groupby(level = 0)
    for name, group in by_country:
        country = group.reset_index()
        last_row = country.iloc[-1, :]
        temp = copy.deepcopy(last_row)
        temp['year'] += 1
        temp[new_name] = temp[column_name]
        temp[column_name] = np.nan

        country = country.append(temp, ignore_index = True)
        output = output.append(country)
    return output

In [14]:
region_year_aid = geoaid_df[['year', 'NAME_1', 'usdco', 'fungible', 'nonfungible']].groupby(['NAME_1', 'year']).sum()

region_year_aid_copy = copy.deepcopy(region_year_aid)

region_year_aid_1 = add_lag(region_year_aid_copy, 'usdco', 'lagged_commitments')

region_year_aid_2 = add_lag(region_year_aid_copy, 'fungible', 'lagged_fungible')
region_year_aid_2 = region_year_aid_2[['NAME_1', 'year', 'lagged_fungible']]

region_year_aid_3 = add_lag(region_year_aid_copy, 'nonfungible', 'lagged_nonfungible')
region_year_aid_3 = region_year_aid_3[['NAME_1', 'year', 'lagged_nonfungible']]

region_year_aid = region_year_aid_1.merge(region_year_aid_2, on = ['NAME_1', 'year']).merge(region_year_aid_3, on = ['NAME_1', 'year'])

region_year_aid['commitments_difference'] = region_year_aid['usdco'] - region_year_aid['lagged_commitments']
region_year_aid['log_commitments'] = np.log(region_year_aid['usdco'] )
region_year_aid['log_lagged_commitments'] = np.log(region_year_aid['lagged_commitments'] )

region_year_aid['log_fungible'] = np.log(region_year_aid['fungible'] )
region_year_aid['log_lagged_fungible'] = np.log(region_year_aid['lagged_fungible'] )

region_year_aid['log_nonfungible'] = np.log(region_year_aid['nonfungible'] )
region_year_aid['log_lagged_nonfungible'] = np.log(region_year_aid['lagged_nonfungible'] )


region_year_aid['log_difference'] = np.log(region_year_aid['commitments_difference'] )
region_year_aid['difference_of_logs'] = region_year_aid['log_commitments'] - region_year_aid['log_lagged_commitments']



region_year_casualties = gconflict[['year', 'NAME_1', 'best_est']].groupby(['NAME_1', 'year']).sum()
region_year_casualties = add_lag(region_year_casualties, 'best_est', 'lagged_casualties')

region_year_casualties['casulaties_difference'] = region_year_casualties['best_est'] - region_year_casualties['lagged_casualties']
region_year_casualties['casulaties_log'] = np.log(region_year_casualties['best_est'])
#region_year_casualties['casulaties_log'] = np.log(region_year_casualties['best_est'])
region_year_casualties['casulaties_standardized'] = (region_year_casualties['best_est'] - np.mean(region_year_casualties['best_est'])) / np.var(region_year_casualties['best_est'])
region_year_casualties['lagged_casualties_standardized'] = (region_year_casualties['lagged_casualties'] - np.mean(region_year_casualties['lagged_casualties'])) / np.var(region_year_casualties['lagged_casualties'])

In [15]:
aid_and_conflict = region_year_aid.merge(region_year_casualties,on = ['NAME_1', 'year'])

In [16]:
aid_and_conflict

,NAME_1,year,usdco,fungible,nonfungible,lagged_commitments,lagged_fungible,lagged_nonfungible,commitments_difference,log_commitments,...,log_nonfungible,log_lagged_nonfungible,log_difference,difference_of_logs,best_est,lagged_casualties,casulaties_difference,casulaties_log,casulaties_standardized,lagged_casualties_standardized
0,Addis Abeba,1989,1.497907e+07,0,14979066,NaN,NaN,NaN,NaN,16.522164,...,16.522164,NaN,NaN,NaN,231.0,NaN,NaN,5.442418,-0.000007,NaN
1,Addis Abeba,1990,1.600550e+08,5679934,154375105,1.497907e+07,0.0,14979066.0,1.450760e+08,18.891028,...,18.854896,16.522164,18.792768,2.368864,2233.0,231.0,2002.0,7.711101,0.001769,-0.000007
2,Addis Abeba,1991,1.542379e+06,0,1542379,1.600550e+08,5679934.0,154375105.0,-1.585127e+08,14.248837,...,14.248837,18.854896,NaN,-4.642192,352.0,2233.0,-1881.0,5.863631,0.000101,0.001769
3,Addis Abeba,1995,1.083111e+07,0,10831108,3.631390e+06,0.0,3631389.0,7.199718e+06,16.197933,...,16.197933,15.105126,15.789552,1.092807,4.0,352.0,-348.0,1.386294,-0.000208,0.000101
4,Addis Abeba,1996,1.062976e+08,0,106297625,1.083111e+07,0.0,10831108.0,9.546652e+07,18.481754,...,18.481754,16.197933,18.374286,2.283821,2.0,4.0,-2.0,0.693147,-0.000210,-0.000208
5,Addis Abeba,1997,4.831444e+07,12335511,35978924,1.062976e+08,0.0,106297625.0,-5.798319e+07,17.693241,...,17.398444,18.481754,NaN,-0.788513,3.0,2.0,1.0,1.098612,-0.000209,-0.000210
6,Addis Abeba,2001,2.480674e+07,0,24806735,1.417812e+07,122671.0,14055446.0,1.062862e+07,17.026626,...,17.026626,16.458520,16.179061,0.559415,0.0,3.0,-3.0,-inf,-0.000212,-0.000209
7,Addis Abeba,2002,1.015339e+08,38736655,62797257,2.480674e+07,0.0,24806735.0,7.672718e+07,18.435903,...,17.955422,17.026626,18.155767,1.409278,2.0,0.0,2.0,0.693147,-0.000210,-0.000212
8,Addis Abeba,2005,3.499031e+07,34697,34955607,2.093362e+08,0.0,209336246.0,-1.743459e+08,17.370582,...,17.369589,19.159452,NaN,-1.788871,4.0,2.0,2.0,1.386294,-0.000208,-0.000210
9,Addis Abeba,2006,4.419607e+07,1480218,42715853,3.499031e+07,34697.0,34955607.0,9.205767e+06,17.604146,...,17.570081,17.369589,16.035341,0.233565,2.0,4.0,-2.0,0.693147,-0.000210,-0.000208


In [26]:
result = sm.ols(formula = 'casulaties_standardized ~ lagged_nonfungible', data = aid_and_conflict).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     casulaties_standardized   R-squared:                       0.002
Model:                                 OLS   Adj. R-squared:                 -0.000
Method:                      Least Squares   F-statistic:                    0.8769
Date:                     Tue, 14 Aug 2018   Prob (F-statistic):              0.349
Time:                             14:48:05   Log-Likelihood:                 3471.5
No. Observations:                      550   AIC:                            -6939.
Df Residuals:                          548   BIC:                            -6930.
Df Model:                                1                                         
Covariance Type:                 nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept          -3.146e-05   2.04e-05     -1.539      0.124   -7.16e-05    8.68e-06
lagged_nonfungible -6.892e-13   7.36e-13     -0.936      0.349   -2.13e-12    7.56e-13
==============================================================================
Omnibus:                      653.166   Durbin-Watson:                   1.334
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            47178.207
Skew:                           5.781   Prob(JB):                         0.00
Kurtosis:                      46.875   Cond. No.                     3.02e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.02e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [172]:
four_groups = {'HR_LA': high_receptivity_low_adaptability,
               'LR_HA': low_receptivity_high_adaptability,
               'LR_LA': low_receptivity_low_adaptability,
               'HR_HA': high_receptivity_high_adaptability}


def estimate_relationship(aid_group):
    region_year_aid = aid_group[['year', 'NAME_1', 'usdco']].groupby(['NAME_1', 'year']).sum()

    region_year_aid = add_lag(region_year_aid, 'usdco', 'lagged_commitments')

    region_year_aid['commitments_difference'] = region_year_aid['usdco'] - region_year_aid['lagged_commitments']
    region_year_aid['log_commitments'] = np.log(region_year_aid['usdco'] )
    region_year_aid['log_lagged_commitments'] = np.log(region_year_aid['lagged_commitments'] )
    region_year_aid['log_difference'] = np.log(region_year_aid['commitments_difference'] )
    region_year_aid['difference_of_logs'] = region_year_aid['log_commitments'] - region_year_aid['log_lagged_commitments']

    aid_and_conflict = pd.merge(region_year_aid, region_year_casualties, on = ['NAME_1', 'year'])

    result = sm.ols(formula = 'best_est ~ log_lagged_commitments', data = aid_and_conflict).fit()
    print(result.summary())
    



for key, value in four_groups.items():
    print('CURRENT GROUP ', key)
    estimate_relationship(value)

CURRENT GROUP  HR_LA
                            OLS Regression Results                            
Dep. Variable:               best_est   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                   0.01467
Date:                Fri, 03 Aug 2018   Prob (F-statistic):              0.904
Time:                        00:20:23   Log-Likelihood:                -1375.5
No. Observations:                 191   AIC:                             2755.
Df Residuals:                     189   BIC:                             2761.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Interce